In [1]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
import re

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
!git clone https://github.com/ronakdm/input-marginalization.git

fatal: destination path 'input-marginalization' already exists and is not an empty directory.


In [3]:
%%bash
cd input-marginalization
git pull
cd ..

Already up to date.


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')
save_dir = "/content/gdrive/My Drive/input-marginalization"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
import pickle
import numpy as np
import time
import datetime
import random
import torch
import torch.nn as nn

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset

In [6]:
import sys
sys.path.append("input-marginalization")

from utils import generate_dataloaders, train, test
from models import CNN

In [7]:
try:
    from transformers import BertForSequenceClassification, AdamW, BertConfig, get_linear_schedule_with_warmup
except ModuleNotFoundError:
    !pip install transformers
    from transformers import BertForSequenceClassification, AdamW, BertConfig, get_linear_schedule_with_warmup
    

In [8]:
LEARNING_RATE = 1e-4
ADAMW_TOLERANCE = 1e-8
BATCH_SIZE = 10
EPOCHS = 30

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

print("Running on '%s'." % device)

Running on 'cuda'.


In [9]:
train_dataloader, validation_dataloader, test_dataloader = generate_dataloaders(BATCH_SIZE)

6,919 training samples.
  876 validation samples.
1,822 test samples.


In [10]:
import pickle
import torch
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [11]:

model = CNN(in_channels = 300, out_channels = 2, vocab_size = len(tokenizer.vocab), embedding_dim= 100).to(device)
save_filename = "cnn_sst2"
optimizer = AdamW(model.parameters(), lr = LEARNING_RATE, eps = ADAMW_TOLERANCE)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = EPOCHS * BATCH_SIZE * len(train_dataloader))

In [12]:
try:
    train(model, EPOCHS, train_dataloader, validation_dataloader, optimizer, scheduler, save_dir, save_filename, device)
except KeyboardInterrupt:
    print("Graceful Exit")


======== Epoch 1 / 30 ========
Training...
  Batch    40  of    692.    Elapsed: 0:00:00.
  Batch    80  of    692.    Elapsed: 0:00:01.
  Batch   120  of    692.    Elapsed: 0:00:01.
  Batch   160  of    692.    Elapsed: 0:00:01.
  Batch   200  of    692.    Elapsed: 0:00:01.
  Batch   240  of    692.    Elapsed: 0:00:02.
  Batch   280  of    692.    Elapsed: 0:00:02.
  Batch   320  of    692.    Elapsed: 0:00:02.
  Batch   360  of    692.    Elapsed: 0:00:02.
  Batch   400  of    692.    Elapsed: 0:00:02.
  Batch   440  of    692.    Elapsed: 0:00:03.
  Batch   480  of    692.    Elapsed: 0:00:03.
  Batch   520  of    692.    Elapsed: 0:00:03.
  Batch   560  of    692.    Elapsed: 0:00:03.
  Batch   600  of    692.    Elapsed: 0:00:04.
  Batch   640  of    692.    Elapsed: 0:00:04.
  Batch   680  of    692.    Elapsed: 0:00:04.

  Average training loss: 0.77
  Training epcoh took: 0:00:04

Running Validation...
  Accuracy: 0.61
  Validation Loss: 0.66
  Validation took: 0:00:00

===

In [14]:

test(model, test_dataloader, device, save_dir, save_filename)


Testing...
  Accuracy: 0.76
  Test Loss: 0.57
  Test took: 0:00:00
